In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Keras
import keras
from keras import metrics

# File and Folder Management
import os, shutil

# Store history
import json

Using TensorFlow backend.


2.2.5


In [0]:
# Set directory
sml_dir = "/content/drive/My Drive/SML_Project"

In [4]:
!pwd
os.chdir(sml_dir)

/content


In [5]:
!pwd

/content


In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from imutils import paths
import argparse
import imutils
import cv2
import os
import gc
import random
from sklearn.decomposition import PCA
from sklearn import model_selection
import pickle
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from datetime import datetime

PCA + Logistic Regression

In [0]:
main_data_folder = '/content/drive/My Drive/SML_Project/binary_data_4'
root_folder='/content/drive/My Drive/SML_Project/binary_data_4'

In [0]:
def image_to_feature_vector(image, size=(500, 500)):
    # resize the image to a fixed size, then flatten the image into a list of raw pixel intensities
    return cv2.resize(image, size).flatten()

# create the list of images in the form of vector
def create_list(root_folder):
    # initialize the raw pixel intensities matrix, and labels list
    rawImages = []
    labels = []
    for subfolder in next(os.walk(root_folder))[1]:
        folder_path=root_folder+'/'+subfolder
        imagePaths = list(paths.list_images(folder_path))
        label=int(subfolder.split('_')[1])
        for (i, imagePath) in enumerate(imagePaths):
            # load the image
            image = cv2.imread(imagePath)
            #label = df[df['image']==imagePaths[i].split('\\')[1].split('.')[0]]['level'].values[0]
            # extract raw pixel intensity features
            pixels = image_to_feature_vector(image)
            rawImages.append(pixels)
            labels.append(label)
    return rawImages,labels


In [0]:
#train data
trainRI,trainLabels=create_list(root_folder + '/train')

In [0]:
#validatioin data
valRI,valLabels=create_list(root_folder + '/val')

In [0]:
#test data
testRI,testLabels=create_list(root_folder + '/test')

In [0]:
import pickle as pk
pk.dump(pca_train, open("pca.pkl","wb"))

In [0]:
#train and transform pca
pca_train = PCA(n_components=100)
pca_train_result = pca_train.fit_transform(X=trainRI)

In [0]:
pca_val_result = pca_train.fit_transform(X=valRI)

In [0]:
pca_test_result = pca_train.fit_transform(X=testRI)

In [0]:
model_logistic = LogisticRegression(n_jobs=8, verbose = 1, random_state=0, solver ='sag')# solver = 'lbfgs'
model_logistic.fit(pca_train_result, trainLabels)

print('Training Results:')
predicted = model_logistic.predict(pca_train_result)
print('Accuracy: ',accuracy_score(trainLabels, predicted))

print('Validation Results:')
predicted = model_logistic.predict(pca_val_result)
print('Accuracy: ',accuracy_score(valLabels, predicted))

print('Testing Results:')
predicted = model_logistic.predict(pca_test_result)
print('Accuracy: ',accuracy_score(testLabels, predicted))


In [0]:
#ROC curve
y_logistic2_score = model_logistic.predict_proba(pca_test_result)
false_positive_rate_log2, true_positive_rate_log2, _=roc_curve(pd.Series(testLabels), y_logistic2_score[:,1])
roc_auc_log2 = auc(false_positive_rate_log2, true_positive_rate_log2)

In [0]:
#plt.figure()
plt.plot(false_positive_rate_log2, true_positive_rate_log2, color='orange',
         lw=2, label='Logistic Regression ROC curve (area = %0.2f)' % roc_auc_log2)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve - Logistic Regression')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()

In [0]:
def calculate_binary_accuracy(prediction, actual):
  true_positive = 0
  true_negative = 0
  false_positive = 0
  false_negative = 0
  for pred, act in zip(prediction, actual):
    if pred >= .5 and act == 1:
      true_positive += 1
    elif pred < .5 and act == 1:
      false_negative += 1
    elif pred < .5 and act == 0:
      true_negative += 1
    elif pred >= .5 and act == 0:
      false_positive += 1
    else:
      print('Some error', pred, act)
  total = true_positive + true_negative + false_positive + false_negative
  print('Total processes images are', total)
  print(true_positive, true_negative, false_positive, false_negative)
  print('Accuracy is ', (true_positive + true_negative) / total)
  print('Accuracy for class 1 images', true_positive / (true_positive + false_negative))
  print('Accuracy for class 0 images', true_negative / (false_positive + true_negative))
  print('Predicts class 1 a percentage of', (true_positive + false_positive) / total)
  print('Predicts class 0 a percentage of', (true_negative + false_negative) / total)
  # Precision of class 1
  print('Precision of class 1', true_positive / (true_positive + false_positive))
  # Recall of class 1
  print('Recall of class 1', true_positive / (true_positive + false_negative))
calculate_binary_accuracy(predicted.astype(int),testLabels)

KNN

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from imutils import paths
import argparse
import imutils
import cv2
import os

In [19]:
df = pd.read_csv('/content/drive/My Drive/SML_Project/trainLabels.csv',index_col=False)
df.head()

,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1


In [20]:
df.groupby('level').count()

,image
level,
0,25810
1,2443
2,5292
3,873
4,708


In [0]:
def image_to_feature_vector(image, size=(300, 300)):
    # resize the image to a fixed size, then flatten the image into a list of raw pixel intensities
    return cv2.resize(image, size).flatten()
def extract_color_histogram(image, bins=(8, 8, 8)):
    # #D histogram
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,[0, 180, 0, 256, 0, 256])
    if imutils.is_cv2():
        hist = cv2.normalize(hist)
    else:
        cv2.normalize(hist, hist)
        # return the flattened histogram as the feature vector
    return hist.flatten()

In [0]:
# grab the list of images that we'll be describing
imagePaths = list(paths.list_images('/content/drive/My Drive/SML_Project/data_566'))
# and labels list
rawImages = []
features = []
labels = []
# loop over the input images
for (i, imagePath) in enumerate(imagePaths):
    # load the image and extract the class label (assuming that our
    # path as the format: /path/to/dataset/{class}.{image_num}.jpg
    image = cv2.imread(imagePath)
    label = df[df['image']==imagePaths[i].split('\\')[1].split('.')[0]]['level'].values[0]
    # extract raw pixel intensity "features", followed by a color histogram
    pixels = image_to_feature_vector(image)
    hist = extract_color_histogram(image)
    rawImages.append(pixels)
    features.append(hist)
    labels.append(label)

In [0]:
# partition the data into training and testing splits, using 75% for training and the remaining 25% for testing
(trainRI, testRI, trainRL, testRL) = train_test_split(rawImages, labels, test_size=0.25, random_state=42)
(trainFeat, testFeat, trainLabels, testLabels) = train_test_split(features, labels, test_size=0.25, random_state=42)


In [0]:
# train and evaluate a k-NN classifer on the raw pixel intensities
model = KNeighborsClassifier(n_neighbors=5,n_jobs=4)
model.fit(trainRI, trainRL)
acc = model.score(testRI, testRL)
print("raw pixel accuracy: {:.2f}%".format(acc * 100))


In [0]:
# train and evaluate a k-NN classifer on the histogram representations
model = KNeighborsClassifier(n_neighbors=5,n_jobs=4)
model.fit(trainFeat, trainLabels)
acc = model.score(testFeat, testLabels)
print("histogram accuracy: {:.2f}%".format(acc * 100))